[Using GPS to accurately establish True Airspeed (TAS)](https://www.kilohotel.com/rv8/rvlinks/doug_gray/TAS_FNL4.pdf), Doug Gray, 1998

In [ ]:
# preamble
import numpy as np
import numpy.linalg as la

def cartesian(rho, theta):
    x = rho * np.cos(theta)
    y = rho * np.sin(theta)
    return x, y

def polar(v):
    x, y = v
    rho = np.hypot(x, y)
    theta = np.arctan2(y, x)
    return rho, theta
    
def fromCompass(speed, track):
    theta = np.radians(90 - track)
    return cartesian(speed, theta)

def toCompass(v):
    speed, theta = polar(v)
    track = (90 - np.degrees(theta)) % 360
    return speed, track

def mag(v):
    return np.sqrt(v.dot(v))
    
def angleBetween(u, v):
    return np.arccos(np.dot(u, v) / (mag(u) * mag(v)))

def angle(v):
    x, y = v
    return np.arctan2(y, x)

In [ ]:
# flight test data
gs1 = fromCompass(140, 192)
gs2 = fromCompass(112, 283)
gs3 = fromCompass(120, 20)

In [ ]:
# Calculation
# Points a, b, c are the tails of gs1, gs2, gs3 respectively
ab = np.subtract(gs1, gs2)
ac = np.subtract(gs1, gs3)
bc = np.subtract(gs2, gs3)
abc = angleBetween(ab, bc)
tas = (mag(ac) / 2) / np.sin(abc)
p = cartesian(tas * np.cos(abc), angle(ac) + np.radians(90))
tas1 = np.subtract(np.divide(ac, 2), p)
w = np.subtract(gs1, tas1)
tas2 = np.subtract(gs2, w)
tas3 = np.subtract(gs3, w)

Doug's results:
- TAS: 130
- Headings: 200°, 287.8°, 11.7°
- Wind: (20.6, 314.8°)


In [ ]:
print("Our results:")
print(f" TAS: {tas}")
print(f" Headings: {[theta for rho, theta in [toCompass(tas1), toCompass(tas2), toCompass(tas3)]]}")
print(f" Wind: {toCompass(-w)}")